In [1]:
import time

print("Last updated : {}".format(time.strftime("%Y.%m.%d %H:%M:%S")))

Last updated : 2018.02.24 12:01:58


# TensorFlow 이름 의미

+ (1) Tensor : n차원 데이터 배열
+ (2) Flow : 텐서의 연산을 실행하는 것

# TensorFlow 기본 원리

+ (1) 그래프 준비하기 (construction) == 파라미터 업데이트할 준비
+ (2) 그래프 실행하기 (execution) == 파라미터 업데이트

---

# MNIST 기본 세팅

**필요한 모듈 임포트**

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

**tf 버전 확인**

In [2]:
tf.__version__

'1.3.0'

**mnist 데이터 다운로드**

In [3]:
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [4]:
print(len(mnist.train.images[0])) # 784
print(len(mnist.train.labels[0])) # 10

784
10


---

# 1) Softmax classifier

## Part I. 그래프 그리기

+ 계산 그래프 그림

![c-mnist-tensorflow_1.jpg](https://github.com/datalater/cs231n-2017/blob/master/images/c-mnist-tensorflow_1.jpg?raw=true)

![c-mnist-tensorflow_2.jpg](https://github.com/datalater/cs231n-2017/blob/master/images/c-mnist-tensorflow_2.jpg?raw=true)

**00 random seed**

In [5]:
tf.set_random_seed(777)  # reproducibility

**01 placeholder**

In [6]:
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

**02 predictions**

In [7]:
W = tf.Variable(tf.random_normal(shape=[784, 10]), name="W_softmax")
b = tf.Variable(tf.random_normal(shape=[10], name="b_softmax"))

predictions = tf.matmul(X, W) + b

# 2) Neural Networks

## Part I. 그래프 그리기

+ 계산 그래프 그림

![](../images/c-mnist-tensorflow_3.jpg)

**02 predictions**

In [8]:


W1 = tf.Variable(tf.random_normal(shape=[784, 256]))
b1 = tf.Variable(tf.random_normal(shape=[256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))

predictions = tf.matmul(L2, W3) + b3

---

**03 cost function and optimizer**

In [9]:
cost= tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(cost)

**04 accuracy**

In [10]:
is_correct = tf.equal(tf.argmax(predictions , axis=1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

## Part II. 그래프 실행

+ **epoch={k}** : 파라미터 업데이트에 전체 데이터가 온전히 사용된 횟수가 {k}번이다.
+ **batch size={k}** : 한 번 파라미터 업데이트 할 때마다 사용되는 데이터 개수가 {k}개이다.
+ **number of iterations={k}** : 파라미터 업데이트 횟수가 {k}번이다.
+ 전체 데이터가 1000개이고 배치 사이즈가 {500}이라면 iteration을 {2}번하면 {1} epoch가 된다.

**05 training epoch/batch**

In [11]:
epochs = 15
batch_size = 100

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict={X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))

Epoch: 0001 cost = 47.440767289
Epoch: 0002 cost = 9.031542797
Epoch: 0003 cost = 5.228914962
Epoch: 0004 cost = 3.486831445
Epoch: 0005 cost = 2.944531191
Epoch: 0006 cost = 2.547421501
Epoch: 0007 cost = 1.991937770
Epoch: 0008 cost = 2.117814162
Epoch: 0009 cost = 1.876204857
Epoch: 0010 cost = 1.406720872
Epoch: 0011 cost = 1.248426753
Epoch: 0012 cost = 1.315571920
Epoch: 0013 cost = 0.990933891
Epoch: 0014 cost = 0.850332184
Epoch: 0015 cost = 0.821606261


**06 test accuracy**

In [12]:
feed_dict={X: mnist.test.images, Y: mnist.test.labels}
print("Accuracy:", sess.run(accuracy, feed_dict=feed_dict))

Accuracy: 0.9607


---

## 연습장 

+ `tf.set_random_seed(777)`의 의미

In [13]:
tf.set_random_seed(777)

In [14]:
a = tf.random_uniform([1])
b = tf.random_normal([1])

In [15]:
with tf.Session() as sess1:
    print(sess1.run(a))  # generates 'A1'
    print(sess1.run(a))  # generates 'A2'
    print(sess1.run(b))  # generates 'B1'
    print(sess1.run(b))  # generates 'B2'

[ 0.09821653]
[ 0.945714]
[ 1.5224483]
[-1.11995387]


In [16]:
with tf.Session() as sess2:
    print(sess2.run(a))  # generates 'A1'
    print(sess2.run(a))  # generates 'A2'
    print(sess2.run(b))  # generates 'B1'
    print(sess2.run(b))  # generates 'B2'

[ 0.09821653]
[ 0.945714]
[ 1.5224483]
[-1.11995387]


---

## 연습장

+ cost 구할 때 tf.reduce_mean을 해줘야 하는 이유

In [17]:
cost_before = tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=Y)

In [18]:
cost_before_val = sess.run(cost_before, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

In [19]:
cost = tf.reduce_mean(cost_before)

In [20]:
cost_val = sess.run(cost, feed_dict={X: mnist.test.images, Y: mnist.test.labels})

In [21]:
cost_val

2.7088461

In [22]:
cost_val.shape

()

In [23]:
cost_before_val.shape

(10000,)

---